In [1]:
import os
import numpy as np
import pandas as pd

from scipy.interpolate import (
    LinearNDInterpolator,
    CloughTocher2DInterpolator,
    griddata,
    RBFInterpolator,
    interp1d
)

from mycolorpy import colorlist as mcp
import matplotlib.pyplot as plt

import plotly
import plotly.graph_objects as go

from mylibs import plots, tools, functions as funcs
from mylibs.tools import Experiment, input_path

D_mean = 15000

PATH = "D:\Works\Diplom-work\Experiments"

In [2]:
%matplotlib qt

# Collect

In [5]:
# OCM
M_but = 74.12
M_ocm = 418.4
M_peta = 298.29
M_dmag = 198.12

but_rho=0.7
def to_df(exp: Experiment):
    df = pd.DataFrame()
    k = 1
    df = exp.d.copy(deep=True).rename(columns={"x": "Temperature", "y": "D"})
    df["Viscosity"] = k * (df["Temperature"] + 273.15) / (df["D"] * 0.001)

    df["compound"] = exp.info["compound"]
    df["rho"] = exp.info["rho"]
    df["w_mass"] = exp.info["w"]
    df["w_vol"] = df['w_mass']*df["rho"]/but_rho
    df["D0"] = exp.info["D0"]
    df["E"] = exp.info["E"]
    df = df[
        [
            "compound",
            "rho",
            "w_mass",
            "w_vol",
            "time",
            "Temperature",
            "D",
            "Viscosity",
            "D0",
            "E",
        ]
    ]
    return df


def collect(M_comp, comp_list):
    df = pd.concat(comp_list)
    df["x"] = (
        (df["w_mass"] / M_but)
        / (df["w_mass"] / M_but + (100 - df["w_mass"]) / M_comp)
        * 100
    )
    return df


comp_list = []
for i in range(9):
    file = [f"{PATH}\OCM{i}\{f}" for f in os.listdir(f"{PATH}\OCM{i}") if ".hdf5" in f][
        0
    ]
    comp_list.append(to_df(Experiment.load_hdf5(file)))

ocm = collect(M_ocm, comp_list)

# PETA
comp_list = []
for i in range(5):
    file = [
        f"{PATH}\PETA{i}\{f}" for f in os.listdir(f"{PATH}\PETA{i}") if ".hdf5" in f
    ][0]
    comp_list.append(to_df(Experiment.load_hdf5(file)))
peta = collect(M_peta, comp_list)

# Butanol
but = to_df(Experiment.load_hdf5(f"{PATH}\BUTANOL\data_BUT_up.hdf5"))
but["x"] = 1
but["w_vol"]=1
but['rho']=but_rho

# DMAG
comp_list = []
for i in range(5):
    file = [
        f"{PATH}\DMAG{i}\{f}" for f in os.listdir(f"{PATH}\DMAG{i}") if ".hdf5" in f
    ][0]
    comp_list.append(to_df(Experiment.load_hdf5(file)))
dmag = collect(M_dmag, comp_list)

In [6]:
ocm

,compound,rho,w_mass,w_vol,time,Temperature,D,Viscosity,D0,E,x
0,ocm,1.72,0.00,0.000000,1.0,13.8,831.980284,344.9,5.805356e+13,59372.593500,0.000000
1,ocm,1.72,0.00,0.000000,2.0,13.8,831.980284,344.9,5.805356e+13,59372.593500,0.000000
2,ocm,1.72,0.00,0.000000,3.0,13.8,831.980284,344.9,5.805356e+13,59372.593500,0.000000
3,ocm,1.72,0.00,0.000000,4.0,13.8,830.775912,345.4,5.805356e+13,59372.593500,0.000000
4,ocm,1.72,0.00,0.000000,5.0,13.8,830.775912,345.4,5.805356e+13,59372.593500,0.000000
...,...,...,...,...,...,...,...,...,...,...,...
4781,ocm,1.49,19.99,42.550143,4860.0,13.4,6070.974576,47.2,1.298231e+11,40133.277971,58.512131
4782,ocm,1.49,19.99,42.550143,4861.0,13.4,6058.139535,47.3,1.298231e+11,40133.277971,58.512131
4783,ocm,1.49,19.99,42.550143,4862.0,13.4,6058.139535,47.3,1.298231e+11,40133.277971,58.512131
4784,ocm,1.49,19.99,42.550143,4863.0,13.4,6058.139535,47.3,1.298231e+11,40133.277971,58.512131


In [7]:
all_df = pd.concat([ocm, peta, but, dmag]).reset_index(drop=True)
df = all_df.groupby(by=["compound", "x", "Temperature"]).mean().reset_index()

In [11]:
res = {}
for compound in df["compound"].unique():
    df_comp = df[df["compound"] == compound]
    xs = df_comp["x"].unique()
    rhos = np.array([df_comp[df_comp["x"] == x]["rho"].mean() for x in xs]) * 1000
    print(f"{compound: >10}:", *xs.astype(int))
    print("*" * 10, *rhos.astype(int))
    res[compound] = list(zip(xs.astype(int), rhos.astype(int)))

      dmag: 0 11 22 35 39
********** 1069 1069 1050 1029 1019
       ocm: 0 12 22 31 38 44 49 54 58
********** 1720 1580 1569 1550 1539 1530 1510 1489 1490
      peta: 0 19 30 41 49
********** 1200 1180 1159 1140 1110


In [12]:
res

{'dmag': [(0, 1069), (11, 1069), (22, 1050), (35, 1029), (39, 1019)],
 'ocm': [(0, 1720),
  (12, 1580),
  (22, 1569),
  (31, 1550),
  (38, 1539),
  (44, 1530),
  (49, 1510),
  (54, 1489),
  (58, 1490)],
 'peta': [(0, 1200), (19, 1180), (30, 1159), (41, 1140), (49, 1110)]}

# 1D

In [8]:
# D-T plots
name = "DMAG, Butanol-1"

compound_filter = (df["compound"] == "dmag") | (
    df["compound"] == "butanol"
)  # yapf: disable

# Plot
data = df[compound_filter]
grouped = data.groupby(by=["w_mass"])
colors_list = mcp.gen_color(cmap="turbo", n=len(grouped))

fig, ax = plt.subplots()
for _, group in grouped:
    w = group["w_mass"].mean()
    T = group["Temperature"]
    color = colors_list.pop()

    D = group["D"] / D_mean
    # ax.scatter(
    #     T,
    #     D,
    #     c=color,
    #     s=2,
    # )

    E = group["E"].mean()
    D0 = group["D0"].mean() / D_mean
    D_ols = D0 * np.exp(-E / (8.314 * (T + 273.15)))
    ax.plot(
        T,
        D_ols,
        label=f"{w: 5.2f} %m",
        c=color,
    )

    ax.set_title(f"{name}: Diffusion - T")
    ax.set_xlabel("Temperature [C]")
    ax.set_ylabel("D/D0")
plt.legend()

In [9]:
# D-w_mass plots
name = "DMAG, Butanol-1"
# +' + Butanol-1'

compound_filter = (df["compound"] == "dmag") | (df["compound"] == "butanol")

# Plot
data = df[compound_filter]
temper_list = np.arange(15, 41, 5)
colors_list = mcp.gen_color(cmap="turbo", n=len(temper_list))

fig, ax = plt.subplots()
for T in temper_list:
    cond = (data["Temperature"] >= T - 1) & (data["Temperature"] <= T + 1)
    group = data[cond]
    group = group[["w_mass", "D"]].groupby(by="w_mass").mean().reset_index()

    w = group["w_mass"]
    color = colors_list.pop()

    D = group["D"] / D_mean
    ax.plot(
        w,
        D,
        c=color,
        # s=5,
        label=f"{T: 3.0f} C",
    )

    ax.set_title(f"{name}: Diffusion - w_mass")
    ax.set_xlabel("w_mass [%]")
    ax.set_ylabel("D/D0")
plt.legend()

In [12]:
# D-x plots
name = "DMAG, Butanol-1"
# +' + Butanol-1'

compound_filter = (df["compound"] == "dmag") | (df["compound"] == "butanol")

# Plot
data = df[compound_filter]
temper_list = np.arange(15, 41, 5)
colors_list = mcp.gen_color(cmap="turbo", n=len(temper_list))

fig, ax = plt.subplots()
for T in temper_list:
    cond = (data["Temperature"] >= T - 1) & (data["Temperature"] <= T + 1)
    group = data[cond]
    group = group[["x", "D"]].groupby(by="x").mean().reset_index()

    x = group["x"]
    color = colors_list.pop()

    D = group["D"] / D_mean
    ax.plot(
        x,
        D,
        c=color,
        # s=5,
        label=f"{T: 3.0f} C",
    )

    ax.set_title(f"{name}: Diffusion - x")
    ax.set_xlabel("x [%]")
    ax.set_ylabel("D/D0")
plt.legend()

In [13]:
# D-v plots
name = "DMAG, Butanol-1"
# +' + Butanol-1'

compound_filter = (df["compound"] == "dmag") | (df["compound"] == "butanol")

# Plot
data = df[compound_filter]
temper_list = np.arange(15, 41, 5)
colors_list = mcp.gen_color(cmap="turbo", n=len(temper_list))

fig, ax = plt.subplots()
for T in temper_list:
    cond = (data["Temperature"] >= T - 1) & (data["Temperature"] <= T + 1)
    group = data[cond]
    group = group[["w_vol", "D"]].groupby(by="w_vol").mean().reset_index()

    x = group["w_vol"]
    color = colors_list.pop()

    D = group["D"] / D_mean
    ax.plot(
        x,
        D,
        c=color,
        # s=5,
        label=f"{T: 3.0f} C",
    )

    ax.set_title(f"{name}: Diffusion - w_vol")
    ax.set_xlabel("w_vol [%]")
    ax.set_ylabel("D/D0")
plt.legend()

In [ ]:
# E plots
data = df.copy(deep=True)
tmp = data[data["compound"] == "butanol"]
tmp2 = tmp.copy(deep=True)
data["compound"] = data["compound"].replace({"butanol": "ocm"})
tmp["compound"] = "peta"
tmp2["compound"] = "dmag"
data = pd.concat([data, tmp, tmp2])

grouped = data.groupby(by=["compound"])
fig, ax = plt.subplots()

for _, group in grouped:
    name = group["compound"].unique()[0]
    group = group[["E", "w_mass"]].groupby(by=["w_mass"]).mean().reset_index()
    w = group["w_mass"]
    E = group["E"] / 1000
    ax.plot(
        [w.iloc[0], w.iloc[-1]],
        [E.iloc[0], E.iloc[-1]],
        # label=f"{name}",
    )
    ax.scatter(
        w,
        E,
        label=f"{name}",
    )

    ax.set_title(f"E - fi")
    ax.set_xlabel("fi [%]")
    ax.set_ylabel("E [kJ]")
plt.legend()

In [ ]:
df.loc[df["compound"] == "butanol", "rho"] = 0.8267

In [ ]:
data = df.copy(deep=True)
tmp = data[data["compound"] == "butanol"]
tmp2 = tmp.copy(deep=True)
data["compound"] = data["compound"].replace({"butanol": "ocm"})
tmp["compound"] = "peta"
tmp2["compound"] = "dmag"
data = pd.concat([data, tmp, tmp2])

grouped = data.groupby(by=["compound"])
fig, ax = plt.subplots()

for _, group in grouped:
    name = group["compound"].unique()[0]
    group = group[["rho", "w_mass"]].groupby(by=["w_mass"]).mean().reset_index()
    w = group["w_mass"]
    E = group["rho"]
    # ax.plot(
    #     [w.iloc[0],w.iloc[-1]],
    #     [E.iloc[0],E.iloc[-1]],
    #     # label=f"{name}",
    # )
    ax.scatter(
        w,
        E,
        label=f"{name}",
    )

    ax.set_title(f"rho - fi")
    ax.set_xlabel("fi [%]")
    ax.set_ylabel("rho [g/ml]")
plt.legend()

# 2D

In [13]:
# 2D surface
compound_filter = df["compound"] == "ocm"
# |(df['compound'] == 'butanol')
data = df[compound_filter]

xy = data[["Temperature", "w_mass"]]
z = data["D"]
X = np.linspace(min(xy["Temperature"]), max(xy["Temperature"]))
Y = np.linspace(min(xy["w_mass"]), max(xy["w_mass"]))
X, Y = np.meshgrid(X, Y)
f = LinearNDInterpolator(xy, z)
Z = f(X, Y)
plt.pcolormesh(X, Y, Z)

# 3D

In [ ]:
fig = go.Figure()
fig.update_layout(
    margin=dict(l=0, r=0, b=0, t=0, pad=0),
    scene=dict(
        xaxis=dict(title="Temperature [C]"),
        yaxis=dict(title="fi [%]"),
        zaxis=dict(title="Diffusion"),
    ),
)
quality = 21

# OCM
compound_filter = df["compound"] == "ocm"
# |(df['compound'] == 'butanol')

data = df[compound_filter]
xy = data[["Temperature", "w_mass"]]
x = data["Temperature"]
y = data["w_mass"]
z = data["D"] / D_mean


f = LinearNDInterpolator(xy, z)

x1 = np.linspace(x.min(), x.max(), quality)
y1 = np.linspace(y.min(), y.max(), quality)
X, Y = np.meshgrid(x1, y1)

Z = f(X, Y)
R = Z
fig.add_trace(
    go.Surface(
        name="OCM-2",
        x=x1,
        y=y1,
        z=Z,
        colorscale="Reds",
        surfacecolor=R,
        showscale=False,
        showlegend=True,
        opacity=0.7,
        lightposition=dict(x=50, y=100, z=1.0),
    )
)

# PETA
compound_filter = df["compound"] == "peta"
# |(df['compound'] == 'butanol')
data = df[compound_filter]

xy = data[["Temperature", "w_mass"]]
x = data["Temperature"]
y = data["w_mass"]
z = data["D"] / D_mean

fz = CloughTocher2DInterpolator(xy, z)

x1 = np.linspace(x.min(), x.max(), quality)
y1 = np.linspace(y.min(), y.max(), quality)
X, Y = np.meshgrid(x1, y1)


Z = fz(X, Y)

fig.add_trace(
    go.Surface(
        name="PETA",
        x=x1,
        y=y1,
        z=Z,
        colorscale="Blues",
        surfacecolor=-R,
        showscale=False,
        showlegend=True,
        lightposition=dict(x=50, y=100, z=1.0),
    )
)
fig.show()

## Plots

In [ ]:
folders = [
    f"{PATH}\{i}"
    for i in os.listdir(PATH)
    if os.path.isdir(f"{PATH}\{i}") and "PETA" in i
]
for folder in folders:

    exp = Experiment()
    file = [file for file in os.listdir(folder) if ".hdf5" in file][0]
    exp.load_hdf5(input_path(f"{folder}\{file}"))
    exp.apply(funcs.C_to_K)
    exp.apply(funcs.nu_D)
    exp.apply(funcs.K_to_C)
    # exp.folder='.'
    plots.temperature_plot(
        exp,
        title="Temperature",
        xlabel="Temperature [C]",
        ylabel="Viscosity [cP]",
        # ylabel='Diffusion [m2/s]',
        interactive=True,
        save_folder=exp.folder,
    )

In [ ]:
exp = Experiment()
exp.load_hdf5(input_path())
# exp.apply(funcs.C_to_K)
# exp.apply(funcs.nu_D)
# exp.apply(funcs.K_to_C)

info, result, func = tools.regress(exp)
exp.set_info(**info)
x = np.linspace(13, 42, 100) + 273.15
ols_res = tools.Experiment(
    pd.DataFrame({"x": x, "y": func(x), "time": x * 0}),
    "interpolated",
)

ols_res.apply(funcs.K_to_C)

plots.comparation_plot(
    exp,
    ols_res,
    title="OLS_Diffusion",
    xlabel="Temperature [C]",
    ylabel="D [m2/s]",
    interactive=True,
    save_folder=exp.folder,
)
# plots.temperature_plot(
#     exp,
#     title='Viscosity',
#     xlabel='Temperature [C]',
#     ylabel='Viscosity [cP]',
#     # ylabel='Diffusion [m2/s]',
#     interactive=True,
#     save_folder=exp.folder,
# )